In [95]:
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler
from sklearn.feature_extraction.text import TfidfVectorizer
SEED = 42

import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import random 
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

## CNN-A

In [122]:
#import the train and test files for financial news
fnews_Xtrain = pd.read_csv('data/train/fnews_Xtrain.csv')
fnews_Xtest = pd.read_csv('data/test/fnews_Xtest.csv')
fnews_ytrain = pd.read_csv('data/train/fnews_ytrain.csv')
fnews_ytest = pd.read_csv('data/test/fnews_ytest.csv')

In [123]:
#drop the first column for all the train and test sets
fnews_Xtrain.drop(columns=fnews_Xtrain.columns[0], axis=1, inplace=True)
fnews_ytrain.drop(columns=fnews_ytrain.columns[0], axis=1, inplace=True)
fnews_Xtest.drop(columns=fnews_Xtest.columns[0], axis=1, inplace=True)
fnews_ytest.drop(columns=fnews_ytest.columns[0], axis=1, inplace=True)

In [124]:
#check the respective shape of the train and test sets
print('Xtrain:',fnews_Xtrain.shape, 'ytrain:' ,fnews_ytrain.shape)
print('Xtest:',fnews_Xtest.shape, 'ytest:' ,fnews_ytest.shape)

Xtrain: (3876, 1) ytrain: (3876, 1)
Xtest: (970, 1) ytest: (970, 1)


In [125]:
fnews_Xtrain['News'] = fnews_Xtrain['News'].str.replace("'","")
fnews_Xtrain['News'] = fnews_Xtrain['News'].str.replace(",","")
fnews_Xtrain['News'] = fnews_Xtrain['News'].str.replace("[","")
fnews_Xtrain['News'] = fnews_Xtrain['News'].str.replace("]","")

C:\Users\delis\AppData\Local\Temp\ipykernel_20772\3694005545.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fnews_Xtrain['News'] = fnews_Xtrain['News'].str.replace("[","")
C:\Users\delis\AppData\Local\Temp\ipykernel_20772\3694005545.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fnews_Xtrain['News'] = fnews_Xtrain['News'].str.replace("]","")


In [126]:
fnews_Xtrain['News'][0]

'russia raisio s food division s home market stretch way vladivostok'

In [127]:
type(fnews_Xtrain["News"][0])

str

In [128]:
fnews_Xtrain.head()

,News
0,russia raisio s food division s home market st...
1,operator need learn use device multiple level ...
2,company expects net sale half reach previous y...
3,bridge km long located anasmotet road e new tr...
4,nokia capcom announced resident evil degenerat...


In [130]:
tf_idf_word_vectorizer = TfidfVectorizer(analyzer = "word")
tf_idf_word_vectorizer.fit(fnews_Xtrain)
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(fnews_Xtrain)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(fnews_Xtest)
x_train_tf_idf_word.toarray()

array([[1.]])

In [129]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer =  Tokenizer()
tokenizer.fit_on_texts(fnews_Xtrain)
sequences = tokenizer.texts_to_sequences(fnews_Xtrain)
print("Vocab length", len(tokenizer.word_index) + 1)
max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
print("Max sequence length", max_seq_length)
sequences = pad_sequences(sequences, maxlen = max_seq_length, padding = 'post')

Vocab length 2
Max sequence length 1
